In [1]:
from pdf2image import convert_from_path
#/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf/CXR1.pdf
images = convert_from_path('/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf/CXR1.pdf')
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>]

In [2]:
from byaldi import RAGMultiModalModel

RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
RAG.index(
    input_path="/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf/CXR1.pdf",
    index_name="test", # index will be saved at index_root/index_name/
    store_collection_with_index=False,
    overwrite=True
)
   

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Index exported to .byaldi/test
Index exported to .byaldi/test


{0: '/home/ccig/Desktop/Nuno/rag_data/NLMCXR_pdf/CXR1.pdf'}

In [4]:
#If the cardiac silhouette and mediastinum size are within normal limits, is there evidence of pneumothorax?
text_query = "If the cardiac silhouette and mediastinum size are within normal limits, is there evidence of pneumothorax?"
results = RAG.search(text_query, k=1)
results

[{'doc_id': 0, 'page_num': 1, 'score': 29.75, 'metadata': {}, 'base64': None}]

In [5]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct",
                                                        trust_remote_code=True, torch_dtype=torch.bfloat16).to('cuda:2').eval()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

image_index = results[0]["page_num"] - 1
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": images[image_index],
            },
            {"type": "text", "text": text_query},
        ],
    }
]

In [7]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
   

In [8]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda:2")

In [9]:
generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)


In [10]:
output_text

['No, there is no evidence of pneumothorax.']